<p align="left">
  <img src="https://raw.githubusercontent.com/python35/IINTS-SDK/main/img/iints_logo.png" width="160">
</p>

# Safety & Supervisor
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/02_Safety_and_Supervisor.ipynb)

**Goal:** understand how the Independent Supervisor evaluates risk and overrides doses.

**You will learn:**
- Direct safety evaluation with `IndependentSupervisor`
- How safety reasons appear in a run

**Note:** the supervisor can reduce a dose to zero when glucose is low or falling quickly.

<!-- NOTEBOOK GUIDE START -->
## Notebook Guide
**What you'll learn**
- Configure `SafetyConfig`
- Inspect supervisor decisions
- Understand safety contract checks

**Prereqs**
- Python 3.10+
- `pip install -e .`

**Estimated runtime:** ~2-3 minutes

**Outputs**
- audit trail logs
- safety decision summaries

**Related docs**
- `docs/COMPREHENSIVE_GUIDE.md` (Safety)
<!-- NOTEBOOK GUIDE END -->


In [1]:
from __future__ import annotations
from pathlib import Path
from typing import Optional
import os
import sys
import subprocess


def _find_repo_root() -> Optional[Path]:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


Repo root: /home/runner/work/IINTS-SDK/IINTS-SDK


## Step 1: Direct safety evaluation


In [2]:
from iints.core.supervisor import IndependentSupervisor

supervisor = IndependentSupervisor()
result = supervisor.evaluate_safety(
    current_glucose=65.0,
    proposed_insulin=1.2,
    current_time=0.0,
    current_iob=0.5,
)
result


{'approved_insulin': 0,
 'safety_level': <SafetyLevel.CRITICAL: 'critical'>,
 'actions_taken': ['HYPO_CUTOFF: Glucose below safety cutoff',
  'CRITICAL: Hypoglycemia - insulin limited'],
 'original_insulin': 1.2,
 'insulin_reduction': 1.2,
 'emergency_mode': True,
 'safety_decision': SafetyDecision(original_dose=1.2, final_dose=0, reason='HYPO_CUTOFF: Glucose below safety cutoff; CRITICAL: Hypoglycemia - insulin limited', triggered=True),
 'safety_reason': 'HYPO_CUTOFF: Glucose below safety cutoff; CRITICAL: Hypoglycemia - insulin limited',
 'safety_triggered': True}

## Step 2: Safety report from a short run


In [3]:
import iints
from iints.presets import get_preset
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus

preset = get_preset("baseline_t1d")
scenario = dict(preset["scenario"])
scenario["stress_events"] = [e for e in scenario.get("stress_events", []) if e.get("event_type") != "exercise"]

algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.4, "carb_ratio": 12.0})


In [4]:
from iints.validation import load_patient_config_by_name

patient_config = load_patient_config_by_name(preset["patient_config"]).model_dump()
patient_config.update(
    {
        "glucose_decay_rate": 0.01,
        "basal_insulin_rate": 0.4,
        "insulin_sensitivity": 60.0,
        "initial_glucose": 150.0,
    }
)


In [5]:
outputs = iints.run_simulation(
    algorithm=algorithm,
    scenario=scenario,
    patient_config=patient_config,
    duration_minutes=180,
    time_step=preset["time_step_minutes"],
    seed=7,
    output_dir=None,
)

outputs["safety_report"]


{'total_violations': 14,
 'violation_breakdown': {'safe': 1,
  'warning': 1,
  'critical': 2,
  'emergency': 10},
 'bolus_interventions_count': 13,
 'emergency_mode_active': False,
 'current_iob': 2.043333333333334,
 'recent_violations': [{'level': 'emergency',
   'message': 'PREDICTED_HYPO: 56.5 mg/dL in 30 min; NEGATIVE_TREND_LIMIT: Glucose dropping at -3.14 mg/dL/min; RECOVERY: Emergency mode cleared',
   'glucose': 99.50600956852085,
   'time': 145.0,
   'original_proposed_insulin': 0.03333333333333333,
   'approved_insulin': 0},
  {'level': 'emergency',
   'message': 'PREDICTED_HYPO: 22.5 mg/dL in 30 min; RECOVERY: Emergency mode cleared',
   'glucose': 104.9853867794215,
   'time': 155.0,
   'original_proposed_insulin': 0.03333333333333333,
   'approved_insulin': 0},
  {'level': 'emergency',
   'message': 'PREDICTED_HYPO: 27.3 mg/dL in 30 min; NEGATIVE_TREND_LIMIT: Glucose dropping at -2.76 mg/dL/min; RECOVERY: Emergency mode cleared',
   'glucose': 92.65637602212438,
   'time': 

### Recap
The supervisor logs reductions and safety reasons in the report.
